The Problem:

![Problem Image](./IMG_2301.jpg)

## Imports

In [33]:
import numpy as np

## Codebase

Setting up the Inputs, kernel weights and the output

In [34]:
A = np.arange(30).reshape(2, 3, 5)
W = np.arange(36).reshape(4, 3, 3)

Z = np.zeros((A.shape[0], W.shape[0], A.shape[2] - W.shape[2] + 1))

In [35]:
A

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14]],

       [[15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]]])

In [36]:
W

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]],

       [[27, 28, 29],
        [30, 31, 32],
        [33, 34, 35]]])

In [37]:
Z

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

Our first Conv. will be between\
`A[0:1, :, 0:3]` - 1st Batch's First segment to convolve over.\
`W[0:1, :, 0:1]` - 1st Filter's weights

To convolve, we do a element wise multiplication and sum up the values with the matrices above:

In [38]:
# Step 1
np.multiply(A[0:1, :, 0:3], W[0:1, :, :])

array([[[ 0,  1,  4],
        [15, 24, 35],
        [60, 77, 96]]])

In [16]:
# Step 2
np.multiply(A[0:1, :, 0:3], W[0:1, :, :]).sum()

312

Converting this to tensordot:

Axes with 2 values in tuple means we are trying to reduce across those dimensions\
Ref: https://medium.com/analytics-vidhya/tensordot-explained-6673cfa5697f - Read from `Tensordot with 2D Tensors`\
to get a better understanding of what axes does, or refer to Aparajith's Quick Primer at @753

For most of this assignment you will use the `Tensordot With Parameter Axes = 2`
part of the blog I pointed to

In [24]:
A[0:1, :, 0:3]

array([[[ 0,  1,  2],
        [ 5,  6,  7],
        [10, 11, 12]]])

In [25]:
W[0:1, :, :]

array([[[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]])

In [28]:
np.tensordot(
    A[0:1, :, 0:3], 
    W[0:1, :, :],
    axes =(
        (1, 2), 
        (1, 2)
    )  # type: ignore
)

array([[312]])

We know how to make it work for one frame, now lets see how it reacts to batches too

In [29]:
np.tensordot(
    A[:, :, 0:3], 
    W[:, :, :],
    axes =(
        (1, 2), 
        (1, 2)
    )  # type: ignore
)

array([[ 312,  798, 1284, 1770],
       [ 852, 2553, 4254, 5955]])

every row is a vector for each input, and columns represent the reductions caused by every filter.

Now all that remains is to assign the values appropriately to `Z`

For this we reshape the output from tensordot() as follows:


In [30]:
np.tensordot(
    A[:, :, 0:3], 
    W[:, :, :],
    axes =(
        (1, 2), 
        (1, 2)
    )  # type: ignore
).reshape(A.shape[0], -1, 1)

array([[[ 312],
        [ 798],
        [1284],
        [1770]],

       [[ 852],
        [2553],
        [4254],
        [5955]]])

And assign these values to `Z`

In [31]:
for i in range(A.shape[2] - W.shape[2] + 1):
    Z[:, :, i:i+1] = np.tensordot(
                            A[:, :, i:i+W.shape[2]], 
                            W, 
                            axes=((1, 2), (1, 2))
                        ).reshape(A.shape[0], -1, 1)

Z

array([[[ 312.,  348.,  384.],
        [ 798.,  915., 1032.],
        [1284., 1482., 1680.],
        [1770., 2049., 2328.]],

       [[ 852.,  888.,  924.],
        [2553., 2670., 2787.],
        [4254., 4452., 4650.],
        [5955., 6234., 6513.]]])

Please note that in this tutorial, we are just witnessing sliding of the kernel across one dimension (horizontal), in your codebase it has to traverse both Horizontally and vertically so code accordingly.

EXTRAS: In case you don't like reshaping your `tensordot()` outputs, the below will work as well.

In [32]:
for i in range(A.shape[2] - W.shape[2] + 1):
    Z[:, :, i] = np.tensordot(
                            A[:, :, i:i+W.shape[2]], 
                            W, 
                            axes=((1, 2), (1, 2))
                        )

Z

array([[[ 312.,  348.,  384.],
        [ 798.,  915., 1032.],
        [1284., 1482., 1680.],
        [1770., 2049., 2328.]],

       [[ 852.,  888.,  924.],
        [2553., 2670., 2787.],
        [4254., 4452., 4650.],
        [5955., 6234., 6513.]]])